In [4]:
# Progetto Data Science Lab
## Sentiment Analysis Corona Virus
## Giacomo De Cobbi
## Alessandro Pontini
## Kevin Tranchina

In [4]:
import os 
import pandas as pd
from datetime import datetime, timedelta
import nltk
import re ##regulare expression
import string
from nltk.tokenize import BlanklineTokenizer
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.tokenize import TweetTokenizer
import matplotlib.pyplot as plt
import subprocess
import operator
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\tranchinake\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
# import non obbligatorio
import qgrid

In [3]:
subprocess.Popen(r'explorer /select,"C:\Users\tranchinake\Desktop\DataScienceUniversity\LabBusinessAnalytic"')

# Import Dataset

In [7]:
# imposta il path dove vuoi hai i dati

path = str(r'C:\Users\tranchinake\Desktop\DataScienceUniversity\LabBusinessAnalytic\Psicology\DS_LAB_PSYCOLOGY_2021')
path = path.replace("\\","/")
os.chdir(path)

In [8]:
# Lettura file 
# 33716 righe
# vaccination = pd.read_csv("./dataset/dataset_europa_inghilterra/all_astrazeneca_eu_in_eu_gb.csv")
vaccination = pd.read_csv("./dataset/dataset_europa_inghilterra_08042021/all_astrazeneca_eu_in_eu_gb_10042021.csv")
df = vaccination
df.columns

Index(['id', 'permalink', 'username', 'text', 'date', 'parola_cercata',
       'location', 'area', 'citta', 'raggio'],
      dtype='object')

In [13]:
qgrid.show_grid(df,grid_options = {'forceFitColumns':False,'syncColumnCellResize': False,'highlightSelectedCell': True,'rowHeight': 25},
                show_toolbar=True,
               column_options ={ 'resizable': True,})

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': False, 'forceFitColumns': False, 'def…

In [1]:
# qgrid.get_changed_df()

In [6]:
vaccination.loc[0,'text']

'AstraZeneca Changes Names Continues Use, Vaccine Lies &amp; Only Thing Keeping Mask Illusion Going Is Us via Rokfin https://t.co/AuGzczZSsV #Rokfin'

## Preprocessiong
### Rimozione Hash, rimozione dei tag e i link per intero
#### Esempio con 
'Ciao mi chiamo @Antnio Antonio #Ciao #Home @Nicole https://facebook.com'

In [193]:
esempio = 'Ciao mi chiamo @Giuseppe Antonio #Ciao #Home @Nicole https://facebook.com'
print(esempio)
# @[A-Za-z0-9]+ rimuove tutte le parole(caratteri alfanumerici) che cominciano con @
# #[A-Za-z0-9]+ rimuove tutte le parole(caratteri alfanumerici) che cominciano con #
# [^0-9A-Za-z ] tiene solo i caratteri alfanumerici
# (\w+:\/\/\S+) rimozione link

esempio = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z ])|(\w+:\/\/\S+)","",esempio)

# fa la stessa cosa di sopra, ma più pulito
# esempio = re.sub("(@\w+)| (#\w+)|([^\w ])|(\w+:\/\/\S+)","",esempio)

print(esempio)

# Rimuove anche gli hashtag interi
# esempio = re.sub("(@[A-Za-z0-9]+)| (#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",esempio)

# Rimozione degli spazi in più
esempio = re.sub(' +',' ',esempio)
print(esempio)

Ciao mi chiamo @Giuseppe Antonio #Ciao #Home @Nicole https://facebook.com
Ciao mi chiamo  Antonio Ciao Home  
Ciao mi chiamo Antonio Ciao Home 


#### Applicazione al nosto dataset

In [9]:
# rimozione del cancelletto nell'hashtag 
# rimozione dei link e dei tag

reNoHashNoTagNoLink = "(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"
reNo_HashTAG_NoTagNoLink = "(@[A-Za-z0-9]+)| (#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"
vaccination.loc[:,'text'] = vaccination.loc[:,'text'].apply(lambda x : ''.join(re.sub(reNo_HashTAG_NoTagNoLink,"",x)))
stampa = vaccination.loc[0:10,'text'].apply(lambda x : print(x))

Hi Gail hope your better if you get the OxfordAstraZeneca jab it put both my niece and her husband for two days after that they were fine I hope and pray its only today your down all my best wishes thinking of you      Al Xx 
Well thats the first Covid19 vaccination done and dusted Astra Zeneca Thank You   Penrith Cumbria 
Safe stable sold at cost AstraZenecas vaccine deserves celebration not scorn 
Thats me jabbed AstraZeneca Boys are in town 
Feck knows whats in that AstraZeneca but Im fooked today 
Thats me now with an armful of Astra Zeneca 
I got my AstraZeneca vaccination today It was all very organised and the staff were lovely 
Delighted to have had my first vaccination this morning at St Andrews Community Hospital Thank you  
What ARE they playing at AstraZeneca row blows wide open as EU fails to use 41 of jabs 
 AstraZeneca is an AngloSwedish firm run by a Frenchman The biggest share of investment in its vaccine came from the US and the bulk of its production is happening in 

In [10]:
## lowerCase
# vaccination.text = vaccination.text.str.lower()

## rimozione dei numeri
vaccination.text = vaccination.text.replace(r'\d+','')

## rimozione spazi bianchi
vaccination.text  = vaccination.text.astype(str).str.strip()

## rimozione doppi spazi bianchi
vaccination.text = vaccination.text.replace(' +',' ', regex=True)

# rimozione punteggiatura
# verificato rimuove anche i cancelletti
vaccination.text=vaccination.text.astype(str).str.translate(str.maketrans('','',string.punctuation))

#rimozione \n
vaccination.text = vaccination.text.replace("\n"," ")

#rimozione apice strano
vaccination.text = vaccination.text.replace("'’'"," ")

#### Verifica sul periodo di rilevamento dei dati

In [9]:
vaccination['date'] = pd.to_datetime(vaccination['date'])
vaccination['date'].dt.year.unique()

array([2021], dtype=int64)

#### Tenere solo i tweet posteriori alla data del 31 dicembre 2019
La diffusione della malattia venne comunicata per la prima volta dalle autorità cinesi all'organizzazione mondiale della sanità il 31 dicembre 2019.

In [10]:
vaccination = vaccination[vaccination['date'] > "2020-12-31 00:00:00"]
print(vaccination['date'].dt.year.unique())
vaccination['date']

# solo 1347 dati

[2021 2020]


1564    2021-01-02 21:48:17
1565    2020-12-31 11:28:52
1657    2021-01-03 01:33:12
1658    2021-01-02 20:29:44
1724    2021-01-05 00:08:33
                ...        
33712   2021-03-12 12:52:49
33713   2021-03-12 12:47:59
33714   2021-03-12 12:45:46
33715   2021-03-12 12:43:36
33716   2021-03-12 12:43:15
Name: date, Length: 31809, dtype: datetime64[ns]

In [11]:
# TweetTokenizer
tknzr = TweetTokenizer()
vaccination['tweet_tokenize'] = vaccination['text'].apply(tknzr.tokenize)
vaccination['tweet_tokenize']

0        [Same, folks, said, daikon, paste, could, trea...
1        [While, the, world, has, been, on, the, wrong,...
2        [coronavirus, SputnikV, AstraZeneca, PfizerBio...
3        [Facts, are, immutable, Senator, even, when, y...
4        [Explain, to, me, again, why, we, need, a, vac...
                               ...                        
33712    [BREAKING, The, information, attack, targeting...
33713    [Pity, as, my, personal, preference, would, ha...
33714    [WHO, in, close, contact, with, Russia, on, Sp...
33715    [URGENT, US, targets, Europe, in, smear, campa...
33716    [BREAKING, US, Targets, Europe, in, Smear, Cam...
Name: tweet_tokenize, Length: 33717, dtype: object

In [12]:
# Rimozione Stop Words
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
stop_words += ['im','youre','thats','get','cant','dont','go',
               'going','gonna','like','got', 'come', 'look','see','theres',
               'say','want','wanna','make','yes','right','']


vaccination['tweet_without_stopwords'] = vaccination['tweet_tokenize'].apply(lambda x: [item for item in x if item not in stop_words])
vaccination['tweet_without_stopwords'] 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tranchinake\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0        [Same, folks, said, daikon, paste, could, trea...
1        [While, world, wrong, side, history, year, hop...
2        [coronavirus, SputnikV, AstraZeneca, PfizerBio...
3        [Facts, immutable, Senator, even, ethically, s...
4        [Explain, need, vaccine, whereareallthesickpeo...
                               ...                        
33712    [BREAKING, The, information, attack, targeting...
33713    [Pity, personal, preference, would, AstraZenec...
33714    [WHO, close, contact, Russia, SputnikV, certif...
33715    [URGENT, US, targets, Europe, smear, campaign,...
33716    [BREAKING, US, Targets, Europe, Smear, Campaig...
Name: tweet_without_stopwords, Length: 33717, dtype: object

# Sentiment Analysis
## VADER ( Valence Aware Dictionary for Sentiment Reasoning)
#### VADER  is a model used for text sentiment analysis that is sensitive to both polarity (positive/negative) and intensity (strength) of emotion. It is available in the NLTK package and can be applied directly to unlabeled text data.

#### To calculate the sentimental score of the entire text, Vader scans the text for known sentimental features, modified the intensity and polarity according to the rules, summed up the scores of features found within the text and normalized the final score to (-1, 1) using function:



$$ compound=\frac{x}{\sqrt{x^2 +a}}$$

 

#### In Vader, alpha is set to be 15 which approximates the maximum expected value of x.

In [11]:
vaccination['scores']  = vaccination['text'].apply(lambda review: sid.polarity_scores(review))

In [12]:
vaccination['scores']

0         {'neg': 0.0, 'neu': 0.665, 'pos': 0.335, 'comp...
1         {'neg': 0.0, 'neu': 0.739, 'pos': 0.261, 'comp...
2         {'neg': 0.0, 'neu': 0.521, 'pos': 0.479, 'comp...
3         {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
4         {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
                                ...                        
914757    {'neg': 0.0, 'neu': 0.707, 'pos': 0.293, 'comp...
914758    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
914759    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
914760    {'neg': 0.524, 'neu': 0.476, 'pos': 0.0, 'comp...
914761    {'neg': 0.178, 'neu': 0.701, 'pos': 0.121, 'co...
Name: scores, Length: 914762, dtype: object

In [15]:
# Test
a = 'I love my girl'
print(sid.polarity_scores(a))

#-0,5<x<+0.5 

{'neg': 0.0, 'neu': 0.323, 'pos': 0.677, 'compound': 0.6369}


In [13]:
vaccination['compound']  = vaccination['scores'].apply(lambda score_dict: score_dict['compound'])
vaccination['compound']

0         0.9485
1         0.5574
2         0.7475
3         0.0000
4         0.0000
           ...  
914757    0.7003
914758    0.0000
914759    0.0000
914760   -0.2960
914761   -0.4588
Name: compound, Length: 914762, dtype: float64

#### Aggregazioni e analisi

In [12]:
print(vaccination.columns)
print(vaccination.info())
print('\nDescribe\n')
print(vaccination['user_location'].describe())

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'permalink', 'username', 'text',
       'date', 'parola_cercata', 'location', 'area', 'citta', 'raggio',
       'scores', 'compound'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203303 entries, 0 to 203302
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Unnamed: 0      203303 non-null  int64         
 1   Unnamed: 0.1    203303 non-null  int64         
 2   id              203303 non-null  int64         
 3   permalink       203303 non-null  object        
 4   username        203303 non-null  object        
 5   text            203303 non-null  object        
 6   date            203303 non-null  datetime64[ns]
 7   parola_cercata  203303 non-null  object        
 8   location        203303 non-null  object        
 9   area            203303 non-null  object        
 10  citta           203303 non-null  object        


KeyError: 'user_location'

In [18]:
vaccination = vaccination[vaccination['user_location'].notna()]
#vaccination['user_location2'] = vaccination['user_location'].dropna()
vaccination.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25937 entries, 0 to 33716
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       25937 non-null  int64  
 1   user_name                25937 non-null  object 
 2   user_location            25937 non-null  object 
 3   user_description         24970 non-null  object 
 4   user_created             25937 non-null  object 
 5   user_followers           25937 non-null  int64  
 6   user_friends             25937 non-null  int64  
 7   user_favourites          25937 non-null  int64  
 8   user_verified            25937 non-null  bool   
 9   date                     25937 non-null  object 
 10  text                     25937 non-null  object 
 11  hashtags                 20371 non-null  object 
 12  source                   25929 non-null  object 
 13  retweets                 25937 non-null  int64  
 14  favorites             

In [155]:
print(len(vaccination.loc[vaccination['user_location'].str.contains('England', regex=True, na=False)]))
print(len(vaccination.loc[vaccination['user_location'].str.contains('Italy', regex=True, na=False)]))
print(len(vaccination.loc[vaccination['user_location'].str.contains('France', regex=True, na=False)]))
print(len(vaccination.loc[vaccination['user_location'].str.contains('UK', regex=True, na=False)]))
print(len(vaccination.loc[vaccination['user_location'].str.contains('USA', regex=True, na=False)]))
print(len(vaccination.loc[vaccination['user_location'].str.contains('U.S.A.', regex=True, na=False)]))
print(len(vaccination.loc[vaccination['user_location'].str.contains('America', regex=True, na=False)]))
print(len(vaccination.loc[vaccination['user_location'].str.contains('Dallas', regex=True, na=False)]))
print(len(vaccination.loc[vaccination['user_location'].str.contains('Rome', regex=True, na=False)]))

1096
78
178
559
1194
8
68
55
28


In [159]:
vaccination.loc[vaccination['user_location'].str.contains('Dallas', regex=True, na=False)]

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
110,1338504763235831814,Nicole Osei,"Dallas, TX","@WBAP247NEWS Morning Drive Anchor/Reporter , @...",2016-08-14 23:21:14,741,460,1240,False,2020-12-14 15:22:58,NEW has received its first shipment ofs vaccin...,"['NEW', 'PfizerBioNTech', 'COVID19']",Twitter Web App,1,2,False
116,1338491684544536577,Nicole Osei,"Dallas, TX","@WBAP247NEWS Morning Drive Anchor/Reporter , @...",2016-08-14 23:21:14,741,460,1240,False,2020-12-14 14:31:00,President Trump says the first vaccine in the ...,"['PfizerBioNTech', 'COVID19']",Twitter Web App,2,4,False
136,1338455738520391680,Nicole Osei,"Dallas, TX","@WBAP247NEWS Morning Drive Anchor/Reporter , @...",2016-08-14 23:21:14,741,460,1240,False,2020-12-14 12:08:10,PfizerBioNTechs vaccines are making their way ...,"['PfizerBioNTech', 'COVID19']",Twitter Web App,3,3,False
681,1339676147190263817,Dr Theresa Patton,"Dallas, TX",OB/Gyn. Mother. Wife. Feminist. Daughter. Busi...,2019-12-10 18:20:23,148,227,2085,False,2020-12-17 20:57:38,All five of us at Kessler Womens Healthcare ha...,NaN,Twitter for iPhone,0,2,False
992,1341004016600428545,Nicole Osei,"Dallas, TX","@WBAP247NEWS Morning Drive Anchor/Reporter , @...",2016-08-14 23:21:14,740,460,1244,False,2020-12-21 12:54:06,Modernas vaccine joins Pfizer Biontechs vaccin...,['COVID19'],Twitter Web App,3,3,False
1303,1341723974112342017,Nicole Osei,"Dallas, TX","@WBAP247NEWS Morning Drive Anchor/Reporter , @...",2016-08-14 23:21:14,741,460,1244,False,2020-12-23 12:34:58,NEW Pfizer and BioNTech have reached a second ...,['NEW'],Twitter Web App,3,1,False
2711,1347395205004189697,Sirisha Bheemaneni,Dallas,NaN,2010-09-07 02:47:55,525,87,306,False,2021-01-08 04:10:24,Got my booster zab up,"['PfizerBioNTech', 'mask', 'StaySafeStayHealthy']",Twitter for iPhone,1,1,False
4765,1354436621949685760,Dr. Ted Price of the Dussor/Price Lab,"Dallas, TX",Ted Price and Greg Dussor run the Pain Neurobi...,2015-06-19 17:21:39,1509,1201,8264,False,2021-01-27 14:30:29,My left arm has some,['PfizerBioNTech'],Twitter for iPhone,0,78,False
4775,1354415971449106438,Dr. Ted Price of the Dussor/Price Lab,"Dallas, TX",Ted Price and Greg Dussor run the Pain Neurobi...,2015-06-19 17:21:39,1509,1201,8264,False,2021-01-27 13:08:26,I am about to get in my car to drive to to get...,['COVID19'],Twitter Web App,1,57,False
4995,1355581818544791557,"Susana Lazarte, MD","Dallas, TX",ID/HIV doc interested in LatinX health issues ...,2020-05-31 02:28:39,77,103,81,False,2021-01-30 18:21:05,Got rhinovirus URI 25 wks after 2nd dose of 6 ...,"['PfizerBioNTech', 'CovidVaccine']",Twitter for iPhone,0,2,False


In [162]:
vaccination.loc[vaccination['user_location'].str.contains('Roma', regex=True, na=False)]

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
1078,1343825523286421504,ex Frate Gervaso 🇮🇹,"Roma, Lazio","Polemico ma buono.\nNo immigrazione,no bandier...",2020-03-07 18:30:58,754,1448,12920,False,2020-12-29 07:45:46,Dear mr Trumpstock market is important but mRN...,NaN,Twitter for Android,0,1,False
1104,1342438012228132865,Adrian Pacurariu,"Timişoara, Romania","Sudent at Polytehnic University of Timisoara, ...",2010-12-30 18:20:50,436,275,58,False,2020-12-25 11:52:18,BREAKING New against by just arrived in on the,"['vaccine', 'COVID19', 'PfizerBioNTech', 'Roma...",Twitter Web App,0,1,False
2566,1347091162394451968,Gea Ciccarelli,"Roma, Lazio",Medical Doctor,2020-12-22 09:28:35,1,23,3,False,2021-01-07 08:02:15,I got the vaccine,"['COVID', 'VaccinoAntiCovid', 'VaccineDay', 'P...",Twitter for Android,0,0,False
3265,1349106279629934593,Alison Mutler,"Bucharest, Romania",Romania-based journalist.,2010-03-01 23:37:45,1694,921,3146,False,2021-01-12 21:29:36,More vaccines please Romanian PM asks European...,"['PfizerBioNTech', 'Romania']",Twitter Web App,0,4,False
3375,1349406457691840514,Gennaro Fabiano,"Cotignola, Emilia Romagna",Arrhythmology and Cardiac Electrophysiology Sp...,2011-10-19 07:52:41,555,1978,123,False,2021-01-13 17:22:24,People often call things impossible that they ...,"['PfizerBioNTech', 'Covid_19', 'stopCovid']",Twitter for iPhone,0,1,False
3420,1349300049268920320,Vincenzo Nasca,Roma,Med student at @unicatt Policlinico Gemelli IR...,2020-08-30 15:32:55,48,193,580,False,2021-01-13 10:19:35,Happy to receive 1st dose of vaccine Thanks to...,['PfizerBioNTech'],Twitter for iPhone,0,5,False
3826,1350383517763567617,brad🌻,"Emilia Romagna, Italia",DONATE:\nhttps://t.co/V1LvgZ4MyX FEDERICO\nhtt...,2013-02-24 15:20:53,900,1755,32278,False,2021-01-16 10:04:54,Un Futuro per Melissa,"['VampireDiaries', 'TheVampireDiaries', 'IOAPR...",Twitter for Android,0,0,False
4249,1352342619993616385,Alison Mutler,"Bucharest, Romania",Romania-based journalist.,2010-03-01 23:37:45,1694,923,3175,False,2021-01-21 19:49:40,PfizerBioNTech Delay in Pfizer vaccine shipmen...,"['PfizerBioNTech', 'vaccine', 'delays', 'EU', ...",Twitter Web App,1,4,False
6258,1359059428633436160,Necula Mihai-Florin,Romania,"Severe weather & natural events+ Economy, Ener...",2014-09-09 22:01:45,2651,158,5625,False,2021-02-09 08:39:52,B1351 is the newPast infection with offers no ...,"['B1351', 'Covid20', 'COVID19']",Twitter for Android,0,0,False
6781,1359243675176480776,SERGIO D'ALESSANDRO,"Roma, Italia.",International Relations - Oil & Gas Geopolitic...,2010-11-03 20:23:04,956,1903,15928,False,2021-02-09 20:52:00,After fake news false statements and a bad cam...,['SputnikV'],Twitter Web App,0,0,False


In [134]:
vaccination['hashtags']

0                                       ['PfizerBioNTech']
1                                                      NaN
2        ['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...
3                                                      NaN
4           ['whereareallthesickpeople', 'PfizerBioNTech']
                               ...                        
33712                       ['BREAKING', 'SputnikV', 'US']
33713                 ['AstraZeneca', 'vaccine', 'Pfizer']
33714                        ['WHO', 'Russia', 'SputnikV']
33715                                           ['URGENT']
33716             ['BREAKING', 'US', 'Russia', 'SputnikV']
Name: hashtags, Length: 33717, dtype: object

In [149]:
vaccination.loc[vaccination['user_location'].str.contains('USA', regex=False, na=False)]

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
38,1337733927885303808,KHuntson,USA🇺🇸,"RN, health educator, writer, researcher, wife,...",2014-01-31 01:25:24,2035,1672,6734,False,2020-12-12 12:19:56,FACT SHEET FOR HEALTHCARE PROVIDERS ADMINISTER...,"['PfizerBioNTech', 'CovidVaccine']",Twitter Web App,0,0,False
41,1337732996380696578,KHuntson,USA🇺🇸,"RN, health educator, writer, researcher, wife,...",2014-01-31 01:25:24,2035,1672,6734,False,2020-12-12 12:16:14,FACT SHEET FOR RECIPIENTS AND CAREGIVERSPfizer...,"['PfizerBioNTech', 'CovidVaccine']",Twitter Web App,0,0,False
51,1338631874814619653,Dr. Taha Khan,India | USA,MD/MPH • PGY1 Peds/Child Neurology @theBCRP (@...,2013-12-30 08:51:23,855,3046,8236,False,2020-12-14 23:48:04,Ive never been so excited to get a vaccine Cov...,"['CovidVaccine', 'PfizerBioNTech', 'VaccinesSa...",Twitter for iPhone,1,10,False
57,1338612922600333315,Kris LaGrange,"LI, NYC, NJ, PA, NE, USA",UCOMM Blog aggregates activist news you can us...,2011-01-27 21:42:20,24686,6077,2994,False,2020-12-14 22:32:45,Members of the delivered the first this mornin...,"['COVID19Vaccine', 'teamsters', 'UPS', 'COVID1...",Twitter Web App,8,18,False
79,1338572466898100224,AcceGen,"New Jersey, USA",AcceGen developed significant expertise in cel...,2017-08-28 07:03:43,12,63,19,False,2020-12-14 19:52:00,How the Vaccine WorksThis article made a detai...,"['PfizerBioNTech', 'COVID19']",Twitter Web App,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32971,1372567686518034436,Jared Mendoza,"Nueva York, USA",Hola,2016-08-18 19:28:47,24,106,40,False,2021-03-18 15:16:52,Son falsas lo acaba de decir la compaa que las...,NaN,Twitter for Android,0,0,False
32977,1372565338622025731,Zvac Salarzai🇦🇫,"California, USA","@SJSU, Humanist, booklover, student of knowled...",2010-01-16 05:18:12,130,713,452,False,2021-03-18 15:07:32,Russians invited d TLB terrorists some parties...,NaN,Twitter for iPhone,0,2,False
33070,1372274994986823693,Chloë M. Smith PhD.,"Maine, USA",#Intersex⚧ • #Disabled with #SEID/#CFS & #Dysa...,2008-04-30 04:41:43,1558,1164,2242,False,2021-03-17 19:53:49,Unthinkable EU Considers Getting a Vaccine Boo...,"['EU', 'Russia', 'SputnikV', 'vaccine']",Twitter for iPad,0,0,False
33251,1371814430720282626,2 Witnesses of Gvt. Racketeering testify-Rev.11:3,"Detroit, Michigan - USA",GOD'S 125 TRILLION$ 2 WITNESSES LITERALLY\nSTA...,2016-10-09 05:56:34,77,1382,65,False,2021-03-16 13:23:42,911THE 1 WORLD ORDER CONSPIRES TO DEPOPULATE V...,NaN,Twitter Web App,0,0,False


In [15]:
vaccination.to_csv("./dataset/dataset_europa_inghilterra_08042021/all_astrazeneca_eu_in_eu_gb_10042021-vader.csv",index=False)